# CSCI 4253 / 5253 - Lab #3 - Patent Problem with SQL - SOLUTION
<div>
 <h2> CSCI 4283 / 5253 
  <IMG SRC="https://www.colorado.edu/cs/profiles/express/themes/cuspirit/logo.png" WIDTH=50 ALIGN="right"/> </h2>
</div>

In this assignment, we're going to solve a problem for which you'll also see the solution in Hadoop and then implement a PySpark solution. We have two databases
* One contains information about patents
* One contains information about patent citations (one patent citing the work of another)
The problem we're going to solve is augmenting the original patent data to include the number of *co-state citations*. In other words, if patent X was issued to someone in Colorado and patent Y was also issued to someone in Colorado and X cites Y, then this is a co-state citation.

The easiest way to solve this is to build an intermediary product from the patent citations table. That table contains `CITING` and `CITED` columns; you would augment it by adding `CITING_STATE` and `CITED_STATE`. Then, it becomes fairly simple to filter out all the cases where those states don't match. You can then use an sql `GROUP BY` to `COUNT(*)` all the co-state citations for a given patent and then join that with the original patents table resulting in an augmented table.

The final results for the first 13 words, sorted in descending order by co-state citations looks like the following
![this final output](final-output.png)

The challenge is going to be that I want you do to this *a single SQL query*. We're assuming you've learned some SQL in a previous life and if not, [now is a great time to learn](https://www.sqlitetutorial.net/). Even if you've done basic SQL, you'll probably need to review [using `select` in a where-clause or using multiple joins](https://dba.stackexchange.com/questions/33553/using-select-in-the-where-clause-of-another-select).


## Logistics

We're going to be using the SQLite3 system which runs entirely from a file (no server needed). The `Makefile` contains commands to download the raw data as ZIP files.

We can run shell commands in our notebook using [builtin "magic" commands](https://ipython.readthedocs.io/en/stable/interactive/magics.html). You need to run this at least once prior to starting the lab to make certain you have the files and have created the `patents.sq3` database file. It will take a few minutes to complete & you should see that the `patents.sq3` file is about 645MBytes in size.

In [1]:
# Temporary fix to CSEL version conflict issue
%pip install --upgrade --user ipython-sql==0.5.0

Note: you may need to restart the kernel to use updated packages.


In [2]:
%%bash
make
rm patents.sq3
zcat < acite75_99.zip | sqlite3 patents.sq3 ".mode csv" ".import /dev/stdin citations"
zcat < apat63_99.zip | sqlite3 patents.sq3 ".mode csv" ".import /dev/stdin patents"
ls -l patents.sq3

make: Nothing to be done for 'all'.


/dev/stdin:2563143: INSERT failed: disk I/O error
/dev/stdin:2563144: INSERT failed: disk I/O error
/dev/stdin:2563145: INSERT failed: disk I/O error
/dev/stdin:2563146: INSERT failed: disk I/O error
/dev/stdin:2563147: INSERT failed: disk I/O error
/dev/stdin:2563148: INSERT failed: disk I/O error
/dev/stdin:2563149: INSERT failed: disk I/O error
/dev/stdin:2563150: INSERT failed: disk I/O error
/dev/stdin:2563151: INSERT failed: disk I/O error
/dev/stdin:2563152: INSERT failed: disk I/O error
/dev/stdin:2563153: INSERT failed: disk I/O error
/dev/stdin:2563154: INSERT failed: disk I/O error
/dev/stdin:2563155: INSERT failed: disk I/O error
/dev/stdin:2563156: INSERT failed: disk I/O error
/dev/stdin:2563157: INSERT failed: disk I/O error
/dev/stdin:2563158: INSERT failed: disk I/O error
/dev/stdin:2563159: INSERT failed: disk I/O error
/dev/stdin:2563160: INSERT failed: disk I/O error
/dev/stdin:2563161: INSERT failed: disk I/O error
/dev/stdin:2563162: INSERT failed: disk I/O error


-rw-r--r-- 1 jovyan users 427749376 Sep  5 22:39 patents.sq3


We'll use another "magic" to run SQL queries in notebook cells. The following will load the SQL extension and connect to the `patents.sq3` file.

In [1]:
%load_ext sql
%sql sqlite:///patents.sq3
%config SqlMagic.style = '_DEPRECATED_DEFAULT'


Following this, we can run individual SQL queries and see the result by putting `%%sql` at the front of a cell. If you don't have that, you'll be running Python code.

So, for example, we can examine our two raw database tables.

In [2]:
%%sql
select * from patents limit 5;

 * sqlite:///patents.sq3
Done.


PATENT,GYEAR,GDATE,APPYEAR,COUNTRY,POSTATE,ASSIGNEE,ASSCODE,CLAIMS,NCLASS,CAT,SUBCAT,CMADE,CRECEIVE,RATIOCIT,GENERAL,ORIGINAL,FWDAPLAG,BCKGTLAG,SELFCTUB,SELFCTLB,SECDUPBD,SECDLWBD
5643451,1997,13696,1996,US,TN,194840,2,12,210,1,19,6,1,1,0,0.5,1,10,0.25,0.1667,0,0
5643452,1997,13696,1994,US,FL,695291,2,16,210,1,19,3,0,1,,0.4444,,12.3333,0,0,,
5643453,1997,13696,1995,FR,,139530,3,11,210,1,19,9,0,1,,0,,12.5556,0,0,,
5643454,1997,13696,1995,GB,,682389,3,7,210,1,19,4,0,1,,0,,20,0,0,,
5643455,1997,13696,1995,AU,,367000,3,3,210,1,19,11,2,1,0.5,0,1,8.2727,0.4545,0.4545,0,0


In [3]:
%%sql
select * from citations limit 5;

 * sqlite:///patents.sq3
Done.


CITING,CITED
3858241,956203
3858241,1324234
3858241,3398406
3858241,3557384
3858241,3634889


If you want to create indexes over various fields, go ahead. It shouldn't affect the correctness of your results but may affect the performance.

## Steps to the full solution

In order to determine when a *cited* patent and a *citing* patent are from the same state,
we're going to need to produce a series of tables that combine information from the citations and the patents tables.

We can use a simple left-outer join (or just **JOIN**) to get the information for one or the other column in the citations table. For example, we can determine the state for *cited* patents using this join:

In [4]:
%%sql
SELECT CITED, patents.POSTATE as CITED_POSTATE, CITING
FROM citations JOIN patents
WHERE CITED==PATENT limit 5;

 * sqlite:///patents.sq3
Done.


CITED,CITED_POSTATE,CITING
5751313,,4021279
5804631,,4312726
5716906,,4584034
5703897,CA,4629457
5727012,NJ,4646478


Then, you'll need to do the same for the `CITING` column as well. As mentioned earlier, you may want to review [using `select` in a where-clause or using multiple joins](https://dba.stackexchange.com/questions/33553/using-select-in-the-where-clause-of-another-select).

## Your solution

Eneter your solution as a single SQL query below:

In [5]:
# Your solution should be in the last cell
"""
Start from patents table.
Join twice - once for citing patent and once for cited
    Want the states
Filter rows where same state
Count co-state citations for each patent
Join these results back into original patents table
"""

'\nStart from patents table.\nJoin twice - once for citing patent and once for cited\n    Want the states\nFilter rows where same state\nCount co-state citations for each patent\nJoin these results back into original patents table\n'

In [8]:
#Steps/debug
#add citing patents state to citation

In [9]:
%%sql
SELECT c.CITING, 
    c.CITED, 
    p.POSTATE AS CITING_POSTSTATE
FROM citations AS c
JOIN patents AS p
    ON c.CITING=p.PATENT
LIMIT 5
        

 * sqlite:///patents.sq3
Done.


CITING,CITED,CITING_POSTSTATE
5643451,4159187,TN
5643451,4268390,TN
5643451,4460468,TN
5643451,4921606,TN
5643451,5188731,TN


In [ ]:
#add cited patents state

In [10]:
%%sql
SELECT c.CITING, 
    c.CITED, 
    p.POSTATE AS CITED_POSTSTATE
FROM citations AS c
JOIN patents AS p
    ON c.CITED=p.PATENT
LIMIT 5
        

 * sqlite:///patents.sq3
Done.


CITING,CITED,CITED_POSTSTATE
4021279,5751313,
4312726,5804631,
4584034,5716906,
4629457,5703897,CA
4646478,5727012,NJ


In [ ]:
#Combine

In [14]:
%%sql
SELECT c.CITING, 
    pciting.POSTATE AS CITING_POSTATE,
    c.CITED, 
    pcited.POSTATE AS CITED_POSTATE
FROM citations AS c
JOIN patents AS pciting
    ON c.CITING=pciting.PATENT
JOIN patents AS pcited
    ON c.CITED=pcited.PATENT
LIMIT 5
        

 * sqlite:///patents.sq3
Done.


CITING,CITING_POSTATE,CITED,CITED_POSTATE
5645501,CA,5647326,
5668639,,5652096,CA
5670060,NY,5804465,NY
5674233,OH,5674503,
5682320,CA,5731683,


In [ ]:
#Filter co-state

In [15]:
%%sql
SELECT c.CITING, 
    pciting.POSTATE AS CITING_POSTATE,
    c.CITED, 
    pcited.POSTATE AS CITED_POSTATE
FROM citations AS c
JOIN patents AS pciting
    ON c.CITING=pciting.PATENT
JOIN patents AS pcited
    ON c.CITED=pcited.PATENT
WHERE pciting.POSTATE=pcited.POSTATE
LIMIT 5

 * sqlite:///patents.sq3
Done.


CITING,CITING_POSTATE,CITED,CITED_POSTATE
5670060,NY,5804465,NY
5689336,,5648658,
5693837,PA,5647900,PA
5697955,MN,5645571,MN
5698182,NJ,5648064,NJ


In [ ]:
#eliminate null cases

In [19]:
%%sql
SELECT c.CITING, 
    pciting.POSTATE AS CITING_POSTATE,
    c.CITED, 
    pcited.POSTATE AS CITED_POSTATE
FROM citations AS c
JOIN patents AS pciting
    ON c.CITING=pciting.PATENT
JOIN patents AS pcited
    ON c.CITED=pcited.PATENT
WHERE pciting.POSTATE=pcited.POSTATE
    AND pciting.POSTATE IS NOT NULL
    AND pcited.POSTATE IS NOT NULL
    AND pciting.POSTATE <>''
    AND pcited.POSTATE <>''
LIMIT 5

 * sqlite:///patents.sq3
Done.


CITING,CITING_POSTATE,CITED,CITED_POSTATE
5670060,NY,5804465,NY
5693837,PA,5647900,PA
5697955,MN,5645571,MN
5698182,NJ,5648064,NJ
5700636,WA,5654418,WA


In [ ]:
#count co state

In [23]:
%%sql
SELECT c.CITED AS patent_id,
    COUNT(*) AS co_state_count
FROM citations AS c
JOIN patents AS pciting
    ON c.CITING=pciting.PATENT
JOIN patents AS pcited
    ON c.CITED=pcited.PATENT
WHERE pciting.POSTATE=pcited.POSTATE
    AND pciting.POSTATE IS NOT NULL
    AND pcited.POSTATE IS NOT NULL
    AND pciting.POSTATE <>''
    AND pcited.POSTATE <>''
GROUP BY c.CITED
ORDER BY co_state_count DESC
LIMIT 13

 * sqlite:///patents.sq3
Done.


patent_id,co_state_count
5651125,24
5678554,16
5673401,15
5671279,15
5729594,14
5694812,14
5706502,13
5696737,13
5692211,13
5689672,13


In [ ]:
#Join

In [25]:
%%sql
SELECT p.PATENT,
    p.POSTATE,
    IFNULL(cs.co_state_count, 0) AS co_state_citations
FROM patents AS p
LEFT JOIN (
    SELECT c.CITED AS patent_id,
        COUNT(*) AS co_state_count
    FROM citations AS c
    JOIN patents AS pciting
        ON c.CITING=pciting.PATENT
    JOIN patents AS pcited
        ON c.CITED=pcited.PATENT
    WHERE pciting.POSTATE=pcited.POSTATE
        AND pciting.POSTATE IS NOT NULL
        AND pcited.POSTATE IS NOT NULL
        AND pciting.POSTATE <>''
        AND pcited.POSTATE <>''
    GROUP BY c.CITED
) AS cs
ON p.PATENT=cs.patent_id
ORDER BY co_state_count DESC
LIMIT 13

 * sqlite:///patents.sq3
Done.


PATENT,POSTATE,co_state_citations
5651125,TX,24
5678554,CA,16
5671279,CA,15
5673401,WA,15
5694812,MI,14
5729594,CA,14
5673399,TX,13
5689672,TX,13
5692211,TX,13
5696737,CA,13


In [27]:
#Only had 3 columns, want whole thing

In [28]:
%%sql
SELECT p.*,
    IFNULL(cs.co_state_count, 0) AS co_state_citations
FROM patents AS p
LEFT JOIN (
    SELECT c.CITED AS patent_id,
        COUNT(*) AS co_state_count
    FROM citations AS c
    JOIN patents AS pciting
        ON c.CITING=pciting.PATENT
    JOIN patents AS pcited
        ON c.CITED=pcited.PATENT
    WHERE pciting.POSTATE=pcited.POSTATE
        AND pciting.POSTATE IS NOT NULL
        AND pcited.POSTATE IS NOT NULL
        AND pciting.POSTATE <>''
        AND pcited.POSTATE <>''
    GROUP BY c.CITED
) AS cs
ON p.PATENT=cs.patent_id
ORDER BY co_state_count DESC
LIMIT 13

 * sqlite:///patents.sq3
Done.


PATENT,GYEAR,GDATE,APPYEAR,COUNTRY,POSTATE,ASSIGNEE,ASSCODE,CLAIMS,NCLASS,CAT,SUBCAT,CMADE,CRECEIVE,RATIOCIT,GENERAL,ORIGINAL,FWDAPLAG,BCKGTLAG,SELFCTUB,SELFCTLB,SECDUPBD,SECDLWBD,co_state_citations
5651125,1997,13717,1995,US,TX,8075,2,28,712,2,22,35,37,1,0.3316,0.6514,1.5946,6.2,0.4545,0.4286,0.5405,0.5405,24
5678554,1997,13808,1996,US,CA,5740,2,33,600,3,32,31,16,1,0.3984,0.6951,1.5625,8.8387,0.129,0.129,0.9333,0.875,16
5671279,1997,13780,1995,US,CA,732559,2,36,705,2,22,0,32,,0.6211,,1.2188,,,,0.0345,0.0313,15
5673401,1997,13787,1995,US,WA,373780,2,28,345,2,23,17,18,1,0.5,0.3945,1.0556,2.8824,0.0588,0.0588,0.6111,0.6111,15
5694812,1997,13857,1995,US,MI,597540,2,42,74,5,59,46,26,0.9783,0.8077,0.8237,2.0385,14.1522,0,0,0.76,0.7308,14
5729594,1998,13955,1996,US,CA,,1,22,379,2,21,4,24,1,0.6146,0.625,0.5833,2.75,,,,,14
5673399,1997,13787,1995,US,TX,280070,2,14,710,2,22,4,18,1,0.2037,0,1.3333,1.5,0.5,0.5,0.2222,0.2222,13
5689672,1997,13836,1993,US,TX,8075,2,38,712,2,22,29,17,1,0.3045,0.5351,3.8824,8.5172,0.1786,0.1724,0.7647,0.7647,13
5692211,1997,13843,1995,US,TX,8075,2,34,712,2,22,32,16,1,0.5781,0.8535,1.125,2.8438,0.0323,0.0313,0.4375,0.4375,13
5696737,1997,13857,1996,US,CA,5740,2,75,367,2,21,23,15,1,0.3911,0.6654,1.6667,9.3913,0.1304,0.1304,0.8667,0.8667,13


In [29]:
#same as above but bettter matches screenshot output

In [30]:
%%sql
SELECT p.*,
    IFNULL(cs.co_state_count, 0) AS CO_CITED_COUNT
FROM patents AS p
LEFT JOIN (
    SELECT c.CITED AS patent_id,
        COUNT(*) AS co_state_count
    FROM citations AS c
    JOIN patents AS pciting
        ON c.CITING=pciting.PATENT
    JOIN patents AS pcited
        ON c.CITED=pcited.PATENT
    WHERE pciting.POSTATE=pcited.POSTATE
        AND pciting.POSTATE IS NOT NULL
        AND pcited.POSTATE IS NOT NULL
        AND pciting.POSTATE <>''
        AND pcited.POSTATE <>''
    GROUP BY c.CITED
) AS cs
ON p.PATENT=cs.patent_id
ORDER BY CO_CITED_COUNT DESC
LIMIT 13

 * sqlite:///patents.sq3
Done.


PATENT,GYEAR,GDATE,APPYEAR,COUNTRY,POSTATE,ASSIGNEE,ASSCODE,CLAIMS,NCLASS,CAT,SUBCAT,CMADE,CRECEIVE,RATIOCIT,GENERAL,ORIGINAL,FWDAPLAG,BCKGTLAG,SELFCTUB,SELFCTLB,SECDUPBD,SECDLWBD,CO_CITED_COUNT
5651125,1997,13717,1995,US,TX,8075,2,28,712,2,22,35,37,1,0.3316,0.6514,1.5946,6.2,0.4545,0.4286,0.5405,0.5405,24
5678554,1997,13808,1996,US,CA,5740,2,33,600,3,32,31,16,1,0.3984,0.6951,1.5625,8.8387,0.129,0.129,0.9333,0.875,16
5671279,1997,13780,1995,US,CA,732559,2,36,705,2,22,0,32,,0.6211,,1.2188,,,,0.0345,0.0313,15
5673401,1997,13787,1995,US,WA,373780,2,28,345,2,23,17,18,1,0.5,0.3945,1.0556,2.8824,0.0588,0.0588,0.6111,0.6111,15
5694812,1997,13857,1995,US,MI,597540,2,42,74,5,59,46,26,0.9783,0.8077,0.8237,2.0385,14.1522,0,0,0.76,0.7308,14
5729594,1998,13955,1996,US,CA,,1,22,379,2,21,4,24,1,0.6146,0.625,0.5833,2.75,,,,,14
5673399,1997,13787,1995,US,TX,280070,2,14,710,2,22,4,18,1,0.2037,0,1.3333,1.5,0.5,0.5,0.2222,0.2222,13
5689672,1997,13836,1993,US,TX,8075,2,38,712,2,22,29,17,1,0.3045,0.5351,3.8824,8.5172,0.1786,0.1724,0.7647,0.7647,13
5692211,1997,13843,1995,US,TX,8075,2,34,712,2,22,32,16,1,0.5781,0.8535,1.125,2.8438,0.0323,0.0313,0.4375,0.4375,13
5696737,1997,13857,1996,US,CA,5740,2,75,367,2,21,23,15,1,0.3911,0.6654,1.6667,9.3913,0.1304,0.1304,0.8667,0.8667,13


In [ ]:
#numbers are off, try to remove the not null section

In [31]:
%%sql
SELECT p.*,
    IFNULL(cs.co_state_count, 0) AS CO_CITED_COUNT
FROM patents AS p
LEFT JOIN (
    SELECT c.CITED AS patent_id,
        COUNT(*) AS co_state_count
    FROM citations AS c
    JOIN patents AS pciting
        ON c.CITING=pciting.PATENT
    JOIN patents AS pcited
        ON c.CITED=pcited.PATENT
    WHERE pciting.POSTATE=pcited.POSTATE
    GROUP BY c.CITED
) AS cs
ON p.PATENT=cs.patent_id
ORDER BY CO_CITED_COUNT DESC
LIMIT 13

 * sqlite:///patents.sq3
Done.


PATENT,GYEAR,GDATE,APPYEAR,COUNTRY,POSTATE,ASSIGNEE,ASSCODE,CLAIMS,NCLASS,CAT,SUBCAT,CMADE,CRECEIVE,RATIOCIT,GENERAL,ORIGINAL,FWDAPLAG,BCKGTLAG,SELFCTUB,SELFCTLB,SECDUPBD,SECDLWBD,CO_CITED_COUNT
5663077,1997,13759,1994,JP,,504585,3,14,438,4,46,7,27,1,0.4225,0.2449,2.2222,4.1429,0,0,0.8519,0.8519,26
5651125,1997,13717,1995,US,TX,8075,2,28,712,2,22,35,37,1,0.3316,0.6514,1.5946,6.2,0.4545,0.4286,0.5405,0.5405,24
5643826,1997,13696,1994,JP,,504585,3,12,438,4,46,15,26,1,0.4704,0.24,2.4615,11.6,0.0769,0.0667,0.8462,0.8462,23
5666914,1997,13773,1996,JP,,402550,3,19,123,5,53,9,21,1,0,0,1.4286,4.4444,0,0,0.1429,0.1429,21
5700333,1997,13871,1996,JP,,504585,3,31,136,4,45,6,20,1,0.475,0.4444,0.35,4.6667,0,0,0.95,0.95,20
5654203,1997,13731,1995,JP,,504585,3,16,438,4,46,6,21,1,0.4444,0,1.3333,6.6667,0.3333,0.3333,0.9048,0.9048,19
5656825,1997,13738,1995,JP,,504585,3,53,257,4,46,2,19,1,0.4211,0,1.2632,4,0,0,0.9444,0.8947,19
5665957,1997,13766,1996,KR,,491065,3,41,250,4,44,19,19,0.8947,0.277,0.7197,1.1053,12.5263,0.0588,0.0526,0.3158,0.3158,19
5646424,1997,13703,1995,JP,,504585,3,31,257,4,46,5,19,1,0.4211,0.32,1.2105,5.6,0,0,0.9474,0.9474,18
5650256,1997,13717,1996,CA,,635240,2,34,430,1,19,7,25,1,0.2848,0,1.76,4.4286,0.5714,0.5714,0.96,0.96,18


In [34]:
%%sql
SELECT c.CITING, pciting.POSTATE, pcited.POSTATE
FROM citations AS c
JOIN patents AS pciting ON c.CITING = pciting.PATENT
JOIN patents AS pcited ON c.CITED = pcited.PATENT
WHERE c.CITED = 5663077;


 * sqlite:///patents.sq3
Done.


CITING,POSTATE,POSTATE_1
5827772,,
5840600,,
5844640,,
5858823,,
5869362,,
5869363,,
5879960,,
5879974,,
5879977,,
5886366,,


In [ ]:
#Final Solution:

In [36]:
%%sql
SELECT p.*,
    IFNULL(cs.co_state_count, 0) AS CO_CITED_COUNT
FROM patents AS p
LEFT JOIN (
    SELECT c.CITED AS patent_id,
        COUNT(*) AS co_state_count
    FROM citations AS c
    JOIN patents AS pciting
        ON c.CITING=pciting.PATENT
    JOIN patents AS pcited
        ON c.CITED=pcited.PATENT
    WHERE pciting.POSTATE=pcited.POSTATE
    GROUP BY c.CITED
) AS cs
ON p.PATENT=cs.patent_id
ORDER BY CO_CITED_COUNT DESC
LIMIT 13

 * sqlite:///patents.sq3
Done.


PATENT,GYEAR,GDATE,APPYEAR,COUNTRY,POSTATE,ASSIGNEE,ASSCODE,CLAIMS,NCLASS,CAT,SUBCAT,CMADE,CRECEIVE,RATIOCIT,GENERAL,ORIGINAL,FWDAPLAG,BCKGTLAG,SELFCTUB,SELFCTLB,SECDUPBD,SECDLWBD,CO_CITED_COUNT
5663077,1997,13759,1994,JP,,504585,3,14,438,4,46,7,27,1,0.4225,0.2449,2.2222,4.1429,0,0,0.8519,0.8519,26
5651125,1997,13717,1995,US,TX,8075,2,28,712,2,22,35,37,1,0.3316,0.6514,1.5946,6.2,0.4545,0.4286,0.5405,0.5405,24
5643826,1997,13696,1994,JP,,504585,3,12,438,4,46,15,26,1,0.4704,0.24,2.4615,11.6,0.0769,0.0667,0.8462,0.8462,23
5666914,1997,13773,1996,JP,,402550,3,19,123,5,53,9,21,1,0,0,1.4286,4.4444,0,0,0.1429,0.1429,21
5700333,1997,13871,1996,JP,,504585,3,31,136,4,45,6,20,1,0.475,0.4444,0.35,4.6667,0,0,0.95,0.95,20
5654203,1997,13731,1995,JP,,504585,3,16,438,4,46,6,21,1,0.4444,0,1.3333,6.6667,0.3333,0.3333,0.9048,0.9048,19
5656825,1997,13738,1995,JP,,504585,3,53,257,4,46,2,19,1,0.4211,0,1.2632,4,0,0,0.9444,0.8947,19
5665957,1997,13766,1996,KR,,491065,3,41,250,4,44,19,19,0.8947,0.277,0.7197,1.1053,12.5263,0.0588,0.0526,0.3158,0.3158,19
5646424,1997,13703,1995,JP,,504585,3,31,257,4,46,5,19,1,0.4211,0.32,1.2105,5.6,0,0,0.9474,0.9474,18
5650256,1997,13717,1996,CA,,635240,2,34,430,1,19,7,25,1,0.2848,0,1.76,4.4286,0.5714,0.5714,0.96,0.96,18
